In [1]:
import os
import glob
import cv2
import tkinter as tk
from tkinter import filedialog, messagebox
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import uniform_filter
from scipy.stats import ttest_rel
from scipy.stats import ttest_ind
from scipy.stats import pearsonr
from scipy.stats import linregress

In [ ]:
# Peripheral Vasoconstriction

### Importing images

myFolder = "mygroup_path"

filePattern = os.path.join(myFolder, "*.bmp")

theFiles = glob.glob(filePattern)

breath_held = []
recovery = []

for fullFileName in theFiles:
    baseName = os.path.basename(fullFileName)
    if "recovery" in baseName:
        recovery.append(fullFileName)
    else:
        breath_held.append(fullFileName)

for fullFileName in breath_held:
    print(f"Now reading (Breath Held): {fullFileName}")
    imageArray = cv2.imread(fullFileName)

for fullFileName in recovery:
    print(f"Now reading (Recovery): {fullFileName}")
    imageArray = cv2.imread(fullFileName)

### LSC calculation function

def compute_lsc(image, window_size):
    image = image.astype(np.float32)
    
    mean = uniform_filter(image, size=window_size)
    mean_sq = uniform_filter(image**2, size=window_size)
    std = np.sqrt(mean_sq - mean**2)
    
    lsc = np.where(mean != 0, std / mean, 0)
    
    return lsc

### Defining Region of Interest (ROI) using Thresholding

# Used threshold of 35 for data analysis
def extract_roi(gray_img, threshold=35):
    blurred = cv2.GaussianBlur(gray_img, (7, 7), 0)

    _, mask = cv2.threshold(blurred, threshold, 255, cv2.THRESH_BINARY)
    
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))
    
    finger_region = cv2.bitwise_and(gray_img, gray_img, mask=mask)
    
    return finger_region, mask

### Alternative method for defining ROI with Manual Selection - ultimately unused

def draw_manual_mask(image):
    clone = image.copy()
    mask = np.zeros_like(image, dtype=np.uint8)

    drawing = False

    def draw_circle(event, x, y, flags, param):
        nonlocal drawing
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            cv2.circle(mask, (x, y), 8, 255, -1)
        elif event == cv2.EVENT_MOUSEMOVE and drawing:
            cv2.circle(mask, (x, y), 8, 255, -1)
        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False

    cv2.namedWindow("Draw Finger ROI (press Q when done)")
    cv2.setMouseCallback("Draw Finger ROI (press Q when done)", draw_circle)

    while True:
        overlay = cv2.addWeighted(clone, 0.7, mask, 0.3, 0)
        cv2.imshow("Draw Finger ROI (press Q when done)", overlay)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    cv2.destroyAllWindows()
    return mask

### Calculating the K_values

bhK = []
window_size = 7

for fullFileName in breath_held:
    print(f"Processing: {fullFileName}")
    
    img = cv2.imread(fullFileName, cv2.IMREAD_GRAYSCALE)

    finger_roi, mask = extract_roi(img)

    lsc_map = compute_lsc(finger_roi, window_size)

    mean_K = np.mean(lsc_map[mask > 0])
    
    print(f"LSC K value (fingers only): {mean_K:.4f}")
    bhK.append(mean_K)

recK = []
window_size = 7

for fullFileName in recovery:
    print(f"Processing: {fullFileName}")
    
    img = cv2.imread(fullFileName, cv2.IMREAD_GRAYSCALE)

    finger_roi, mask = extract_roi(img)

    lsc_map = compute_lsc(finger_roi, window_size)

    mean_K = np.mean(lsc_map[mask > 0])
    
    print(f"LSC K value (fingers only): {mean_K:.4f}")
    recK.append(mean_K)

### Performing pooled analysis 

selected_file_path_updated = 'classdata_path'
df_selected_updated = pd.read_csv(selected_file_path_updated, header=None)

# Start of recovery times for each of the groups
manual_recovery_times = {0: 75, 1: 120, 2: 60, 3: 30}

groups = []
current_group = {"time": [], "k_values": [], "labels": []}

for _, row in df_selected_updated.iterrows():
    if "Time" in str(row[0]) or "K Value" in str(row[0]) or "Values" in str(row[0]):
        if current_group["time"]:
            groups.append(current_group)
            current_group = {"time": [], "k_values": [], "labels": []}
    else:
        try:
            t = float(row[0])
            k = float(row[1])
            label = str(row[2]) if not pd.isna(row[2]) else ""
            current_group["time"].append(t)
            current_group["k_values"].append(k)
            current_group["labels"].append(label.lower())
        except:
            continue

if current_group["time"]:
    groups.append(current_group)

plt.figure(figsize=(10, 6))

for i, group in enumerate(groups):
    times = group["time"]
    k_vals = group["k_values"]
    recovery_time = manual_recovery_times.get(i)

    capped_data = [(t, k) for t, k in zip(times, k_vals) if t <= 270]
    if not capped_data:
        continue

    times_capped, k_vals_capped = zip(*capped_data)

    if recovery_time is not None:
        recovery_index = next((j for j, t in enumerate(times_capped) if t >= recovery_time), len(times_capped))

        color = plt.cm.tab10(i % 10)

        plt.plot(times_capped[:recovery_index + 1], k_vals_capped[:recovery_index + 1],
                 color=color, linewidth=2, label=f'Group {i+1}')

        if recovery_index < len(times_capped):
            plt.plot(times_capped[recovery_index:], k_vals_capped[recovery_index:],
                     linestyle='--', linewidth=2, color=color)
    else:
        plt.plot(times_capped, k_vals_capped, linewidth=2, label=f'Group {i+1}')

plt.xlabel("Time (sec)")
plt.ylabel("K Value")
plt.title("K Values Over Time with Recovery Start")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

groups_2d = np.empty((17, 9))

for i, group in enumerate(groups):
    groups_2d[i] = group['k_values'][:9]

column_means = np.mean(groups_2d, axis=0)
column_stds = np.std(groups_2d, axis=0)

print("Column Means:", column_means)
print("Column STDs:", column_stds)

manual_recovery_times = {0: 75, 1: 120, 2: 60, 3: 30}

t_test_results = []

for i, group in enumerate(groups):
    times = np.array(group["time"])
    k_vals = np.array(group["k_values"])
    recovery_time = manual_recovery_times[i]

    breath_vals = k_vals[times < recovery_time]
    recovery_vals = k_vals[times >= recovery_time]

    t_stat, p_val = ttest_ind(breath_vals, recovery_vals, equal_var=False)

    significance = "Yes" if p_val < 0.05 else "No"

    t_test_results.append({
        "Participant": f"Participant {i+1}",
        "t-statistic": round(t_stat, 4),
        "p-value": round(p_val, 4),
        "Significance": significance
    })

pd.DataFrame(t_test_results)